In [1]:
import gym
import random

In [2]:
env_name = "CartPole-v1"
env = gym.make(env_name) #render_mode="human"

states = env.observation_space.shape[0]
actions = env.action_space.n

### Run the simulation using random actions

In [4]:
env.close()

In [7]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode, score))
    env.close()

Episode:1 Score:25.0


# Create Neural Network

In [8]:
# Neural Network
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# RL Agent
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

2023-01-29 11:56:22.374324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 11:56:22.751806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omaritani/.local/lib/python3.8/site-packages/cv2/../../lib64:/home/omaritani/catkin_ws/devel/lib:/opt/ros/noetic/lib
2023-01-29 11:56:22.751905: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-29 11:56:23.891270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build Agent

In [7]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [11]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

2023-01-17 14:50:00.205926: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_1_2/bias/Assign' id:767 op device:{requested: '', assigned: ''} def:{{{node dense_1_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1_2/bias, dense_1_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [12]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 20000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 20:05 - reward: 1.0000

2023-01-17 14:50:08.556535: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_2/BiasAdd' id:75 op device:{requested: '', assigned: ''} def:{{{node dense_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2/MatMul, dense_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-01-17 14:50:08.613140: W tensorflow/c/c_api.cc:291] Operation '{name:'count_7/Assign' id:954 op device:{requested: '', assigned: ''} def:{{{node count_7/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](count_7, count_7/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or 

10000/10000 [==============================] - 77s 8ms/step - reward: 1.0000
20 episodes - episode_reward: 484.550 [288.000, 500.000] - loss: 10.647 - mae: 62.064 - mean_q: 124.945

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 1.0000
done, took 152.739 seconds


In [13]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
500.0


### Test the Agent 

In [17]:
_ = dqn.test(env, nb_episodes=3, visualize=True)
env.close()

Testing for 3 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500


### Save Model Weights

In [18]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)